In [1]:
import importlib

import model
import pan_loader
import base_config
import loss_functions as L

import utils

import torch
import torch.optim as optim

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

train_img_dir = val_img_dir 
train_seg_dir = val_seg_dir 
train_ann_json = val_ann_json 

In [3]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [4]:
config = base_config.Config()

In [5]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [6]:
net = model.hgmodel()
net = net.cuda()

In [7]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)

set_trainable(net.mb0,True)
set_trainable(net.mb1,True)
set_trainable(net.cb,True)

set_trainable(net.iresnet0, False)
set_trainable(net.iresnet1, False)

# for name,child in net.iresnet0.named_children():
#     if name[:-1] == "layer":
#         [set_trainable(s.ignore_filters,True) for s in child[::2]]
#         [set_trainable(s.copy_filters,True) for s in child[::2]]

param_lr = []
param_lr.append({'params': net.mb0.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.mb1.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.cb.parameters(),'lr':1e-3,'momentum':0.9})

# for name,child in net.iresnet0.named_children():
#     if name[:-1] == "layer":
#         for s in child[::2]:
#             lr = 0
#             if int(name[-1]) > 0:
#                 lr = 1e-3
#             else:
#                 lr = 0
#             param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
#             param_lr.append({'params':s.copy_filters.parameters(),'lr':1e-3,'momentum':0.9})

param_lr = [{'params':net.parameters(),'lr':1e-3, 'momentum':0.9}]
net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
print(param_lr)
optimizer = optim.SGD(param_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=100, min_lr=1e-2, eps=1e-08)

65293298 12326186
[{'params': <generator object Module.parameters at 0x7f1cacccb780>, 'lr': 0.001, 'momentum': 0.9}]


In [ ]:
ckpt = utils.Checkpoint(iters_per_epoch=2, model_dir="./models/", model_name="first")

for i, data in enumerate(train_loader,0):
    print("batch %d:"%i)
    
    optimizer.zero_grad()
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    del(data)
    outs = net([images,impulses])
    del(images, impulses)
#     loss = outs[0].mean()
    loss = L.loss_criterion1(outs, [instance_masks, cat_ids])
    del(instance_masks, cat_ids, outs)
#     ckpt.update(loss.data, net)
    loss.backward()
    del(loss)
    optimizer.step()

batch 0:
torch.Size([12, 3, 448, 448]) torch.Size([12, 1, 448, 448])
torch.Size([12, 66, 112, 112]) torch.Size([12, 132, 56, 56]) torch.Size([12, 264, 28, 28]) torch.Size([12, 528, 14, 14])
torch.Size([12, 66, 112, 112]) torch.Size([12, 132, 56, 56]) torch.Size([12, 264, 28, 28]) torch.Size([12, 528, 14, 14])
torch.Size([12, 1, 448, 448]) torch.Size([12, 1, 448, 448]) torch.Size([1, 12]) torch.Size([12, 121])
batch 1:
torch.Size([10, 3, 448, 448]) torch.Size([10, 1, 448, 448])
torch.Size([10, 66, 112, 112]) torch.Size([10, 132, 56, 56]) torch.Size([10, 264, 28, 28]) torch.Size([10, 528, 14, 14])
torch.Size([10, 66, 112, 112]) torch.Size([10, 132, 56, 56]) torch.Size([10, 264, 28, 28]) torch.Size([10, 528, 14, 14])
torch.Size([10, 1, 448, 448]) torch.Size([10, 1, 448, 448]) torch.Size([1, 10]) torch.Size([10, 121])
batch 2:
torch.Size([4, 3, 448, 448]) torch.Size([4, 1, 448, 448])
torch.Size([4, 66, 112, 112]) torch.Size([4, 132, 56, 56]) torch.Size([4, 264, 28, 28]) torch.Size([4, 528,